In [0]:
# Definição de credenciais e caminhos
from pyspark.sql.functions import year, month, lit, coalesce, col, regexp_extract
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType


aws_access_key_id = "suacredencial"
aws_secret_access_key = "suacredencial"
s3_bucket_name = "ifood-case-data-lake-vitor"

# caminho da landing zone com os arquivos Parquet brutos
landing_zone_path = f"s3a://{s3_bucket_name}/landing_zone/"
# caminho para a camada Bronze em formato Delta
bronze_path = f"s3a://{s3_bucket_name}/bronze_layer/"

read_options = {"fs.s3a.access.key": aws_access_key_id, "fs.s3a.secret.key": aws_secret_access_key}
write_options = {"fs.s3a.access.key": aws_access_key_id, "fs.s3a.secret.key": aws_secret_access_key}

In [0]:
from pyspark.sql.functions import col, coalesce, year, month
from pyspark.sql.types import StringType, IntegerType

# Leitura e salvamento na camada Bronze em formato Parquet
taxi_types = ['yellow', 'green', 'fhv', 'fhvhv']
year_str = '2023'
months = ['01', '02', '03', '04', '05']

for taxi_type in taxi_types:
    df_list = []
    
    for month_str in months:
        filename_prefix = 'fhvhv' if taxi_type == 'fhvhv' else taxi_type
        filename = f"{filename_prefix}_tripdata_{year_str}-{month_str}.parquet"
        source_file_path = f"{landing_zone_path}{filename}"
        
        try:
            df_monthly = spark.read.options(**read_options).parquet(source_file_path)
            # CONVERSÃO PARA STRING: AQUI ESTÁ A LÓGICA QUE RESOLVE O PROBLEMA DE SCHEMA ..000 O TAL DO SCHEMA DRIFT
            for c in df_monthly.columns:
                df_monthly = df_monthly.withColumn(c, col(c).cast(StringType()))
            df_list.append(df_monthly)
        except Exception as e:
            print(f"Erro ao ler o arquivo {filename}: {e}")
            continue
    
    if df_list:
        df_raw_unified = df_list[0]
        for df_monthly in df_list[1:]:
            df_raw_unified = df_raw_unified.unionByName(df_monthly, allowMissingColumns=True)
            
        #Adiciona as colunas de partição de ano e mês antes de salvar
        df_with_partitions = (df_raw_unified
            .withColumn("standard_pickup_datetime", coalesce(
                *[col(c) for c in ["tpep_pickup_datetime", "lpep_pickup_datetime", "pickup_datetime"] if c in df_raw_unified.columns]
            ))
            .withColumn("year", year(col("standard_pickup_datetime")).cast(IntegerType()))
            .withColumn("month", month(col("standard_pickup_datetime")).cast(IntegerType()))
            .filter(col("year") == 2023).filter(col("month").between(1, 5))
            .drop("standard_pickup_datetime")
        )
        
        # Reduz o número de arquivos Parquet para 1 por partição
        df_with_partitions = df_with_partitions.coalesce(1)
        
        destination_path = f"{bronze_path}{taxi_type}/"
        (df_with_partitions.write.options(**write_options)
            .format("parquet")
            .mode("overwrite")
            .partitionBy("year", "month")
            .save(destination_path)
        )
        print(f"Dados brutos de {taxi_type} de todos os meses salvos com sucesso na camada Bronze em: {destination_path}")

Dados brutos de yellow de todos os meses salvos com sucesso na camada Bronze em: s3a://ifood-case-data-lake-vitor/bronze_layer/yellow/
Dados brutos de green de todos os meses salvos com sucesso na camada Bronze em: s3a://ifood-case-data-lake-vitor/bronze_layer/green/
Dados brutos de fhv de todos os meses salvos com sucesso na camada Bronze em: s3a://ifood-case-data-lake-vitor/bronze_layer/fhv/
Dados brutos de fhvhv de todos os meses salvos com sucesso na camada Bronze em: s3a://ifood-case-data-lake-vitor/bronze_layer/fhvhv/


In [0]:
# Contagem de registros por partição (ano e mês)
taxi_types = ['yellow', 'green', 'fhv', 'fhvhv']
total_records = 0

for taxi_type in taxi_types:
    try:
        # Lê a camada Bronze particionada
        df_bronze = spark.read.options(**read_options).parquet(f"{bronze_path}{taxi_type}/")
        
        # Agrupa por ano e mês e conta os registros
        print(f"--- Contagem de registros para {taxi_type} por partição ---")
        df_counts = df_bronze.groupBy("year", "month").count().orderBy("year", "month")
        df_counts.show()
        
        count_total_type = df_counts.agg({"count": "sum"}).collect()[0][0]
        print(f"Tipo de táxi: {taxi_type}, Total de Registros: {count_total_type}\n")
        total_records += count_total_type
        
    except Exception as e:
        print(f"Erro ao processar o tipo de táxi {taxi_type}: {e}\n")

print(f"Total de registros em todos os arquivos da camada Bronze: {total_records}")

--- Contagem de registros para yellow por partição ---
+----+-----+-------+
|year|month|  count|
+----+-----+-------+
|2023|    1|3066726|
|2023|    2|2914003|
|2023|    3|3403660|
|2023|    4|3288248|
|2023|    5|3513645|
+----+-----+-------+

Tipo de táxi: yellow, Total de Registros: 16186282

--- Contagem de registros para green por partição ---
+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2023|    1|68231|
|2023|    2|64792|
|2023|    3|72039|
|2023|    4|65391|
|2023|    5|69168|
+----+-----+-----+

Tipo de táxi: green, Total de Registros: 339621

--- Contagem de registros para fhv por partição ---
+----+-----+-------+
|year|month|  count|
+----+-----+-------+
|2023|    1|1114320|
|2023|    2|1110797|
|2023|    3|1328242|
|2023|    4|1246479|
|2023|    5|1385826|
+----+-----+-------+

Tipo de táxi: fhv, Total de Registros: 6185664

--- Contagem de registros para fhvhv por partição ---
+----+-----+--------+
|year|month|   count|
+----+-----+--------+
|2023|    1|184790